In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import kagglehub

# Download latest version
path = kagglehub.dataset_download("marquis03/plants-classification")
print("Path to dataset files:", path)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Path to dataset files: /kaggle/input/plants-classification


In [2]:
import os, torch, numpy as np, random, matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
import timm, cv2
from tqdm import tqdm
# Set seeds for reproducibility
torch.manual_seed(2024)
np.random.seed(2024)
random.seed(2024)

device = "cuda" if torch.cuda.is_available() else "cpu"

# ---------------- Dataset ----------------
class CustomDataset(Dataset):
    def __init__(self, root, transformations=None):
        self.transformations = transformations
        self.im_paths = sorted(glob(f"{root}/*/*"))
        self.cls_names, self.cls_counts, count = {}, {}, 0
        for path in self.im_paths:
            class_name = self.get_class(path)
            if class_name not in self.cls_names:
                self.cls_names[class_name] = count
                count += 1
                self.cls_counts[class_name] = 1
            else:
                self.cls_counts[class_name] += 1

    def get_class(self, path):
        return os.path.basename(os.path.dirname(path))

    def __len__(self):
        return len(self.im_paths)

    def __getitem__(self, idx):
        im_path = self.im_paths[idx]
        im = Image.open(im_path).convert("RGB")
        gt = self.cls_names[self.get_class(im_path)]
        if self.transformations:
            im = self.transformations(im)
        return im, gt

# ---------------- Transforms ----------------
mean, std, im_size = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225], 300
train_tfs = T.Compose([
    T.Resize((im_size, im_size)),
    T.RandomResizedCrop(im_size, scale=(0.85, 1.0)),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomRotation(degrees=20),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.01),
    T.ToTensor(),
    T.Normalize(mean, std)
])

val_test_tfs = T.Compose([
    T.Resize((im_size, im_size)),
    T.ToTensor(),
    T.Normalize(mean, std)
])

# ---------------- Dataloader ----------------
def get_dataloader(path, transformations, batch_size=32, shuffle=False):
    ds = CustomDataset(root=path, transformations=transformations)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=4)
    return dl, ds.cls_names

# ---------------- Paths ----------------
root = "/kaggle/input/plants-classification"
tr_dl, classes = get_dataloader(os.path.join(root, "train"), train_tfs, shuffle=True)
val_dl, _ = get_dataloader(os.path.join(root, "val"), val_test_tfs)
ts_dl, _  = get_dataloader(os.path.join(root, "test"), val_test_tfs, batch_size=1)

# ---------------- Model: EfficientNet-B3 ----------------
model = timm.create_model("efficientnet_b3a", pretrained=True, num_classes=len(classes))
model.to(device)

# ---------------- Optimizer & Scheduler ----------------
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

# ---------------- Training Loop ----------------
def to_device(batch):
    return batch[0].to(device), batch[1].to(device)

best_loss, threshold, not_improved, patience = float("inf"), 0.01, 0, 10
save_prefix, save_dir = "plant", "saved_models"
tr_losses, val_losses, tr_accs, val_accs = [], [], [], []
epochs = 30

print("Start training...")
for epoch in range(epochs):
    model.train()
    epoch_loss, epoch_acc = 0, 0
    for batch in tqdm(tr_dl):
        ims, gts = to_device(batch)
        preds = model(ims)
        loss = loss_fn(preds, gts)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += (preds.argmax(1) == gts).sum().item()

    scheduler.step()
    tr_loss, tr_acc = epoch_loss / len(tr_dl), epoch_acc / len(tr_dl.dataset)
    tr_losses.append(tr_loss); tr_accs.append(tr_acc)
    print(f"[Epoch {epoch+1}] Train Loss: {tr_loss:.3f}, Train Acc: {tr_acc:.3f}")

    model.eval()
    val_loss, val_acc = 0, 0
    with torch.no_grad():
        for batch in val_dl:
            ims, gts = to_device(batch)
            preds = model(ims)
            loss = loss_fn(preds, gts)
            val_loss += loss.item()
            val_acc += (preds.argmax(1) == gts).sum().item()

    val_loss /= len(val_dl)
    val_acc /= len(val_dl.dataset)
    val_losses.append(val_loss); val_accs.append(val_acc)
    print(f"[Epoch {epoch+1}] Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.3f}")

    if val_loss < best_loss - threshold:
        os.makedirs(save_dir, exist_ok=True)
        best_loss = val_loss
        torch.save(model.state_dict(), f"{save_dir}/{save_prefix}_best_model.pth")
        not_improved = 0
    else:
        not_improved += 1
        if not_improved == patience:
            print("Early stopping.")
            break



/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name efficientnet_b3a to current efficientnet_b3.
  model = create_fn(


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

Start training...


100%|██████████| 657/657 [04:22<00:00,  2.50it/s]

[Epoch 1] Train Loss: 1.182, Train Acc: 0.742


...

[Epoch 24] Train Loss: 0.410, Train Acc: 0.964


[Epoch 24] Val Loss: 0.547, Val Acc: 0.919


100%|██████████| 657/657 [04:21<00:00,  2.51it/s]

[Epoch 25] Train Loss: 0.409, Train Acc: 0.963


[Epoch 25] Val Loss: 0.543, Val Acc: 0.922
Early stopping.
